In [1]:
import collections
import copy
import datetime
import itertools
import sys

import json
import pathlib
import re
from pprint import pprint

import nest_asyncio
nest_asyncio.apply()
import texttable

import pytourney
import pyfumbbl
import cibblbibbl

In [2]:
show_Ids = True
G = cibblbibbl.CIBBL
Achievement = cibblbibbl.achievement.Achievement
Team = cibblbibbl.team.Team
player = cibblbibbl.player.player
Match = cibblbibbl.match.Match
Matchup = cibblbibbl.matchup.Matchup
Replay = cibblbibbl.replay.Replay
Tournament = cibblbibbl.tournament.handler.default.Tournament
NormalPlayer = cibblbibbl.player.NormalPlayer
StarPlayer = cibblbibbl.player.StarPlayer

In [3]:
G.init()

In [4]:
standings = max(G.seasons).prestigestandings()
standings

[[1, 1095, (210, 170, 90, 80, 95, 105), Team(867862)],
 [2, 1020, (150, 130, 220, 40, 85, 95), Team(903654)],
 [3, 995, (230, 60, 270, 30, 70, 25), Team(874452)],
 [4, 990, (70, 160, 200, 100, 85, 65), Team(876774)],
 [5, 975, (260, 120, 130, 20, 75, 140), Team(886114)],
 [6, 970, (180, 120, 230, 25, 80, 115), Team(895929)],
 [7, 935, (240, 140, 170, 80, 70, 30), Team(874966)],
 [8, 870, (120, 150, 130, 25, 85, 115), Team(892248)],
 [9, 860, (220, 220, 60, 30, 60, 80), Team(879131)],
 [10, 855, (160, 140, 150, 55, 85, 105), Team(878945)],
 [11, 830, (170, 150, 160, 65, 95, 90), Team(874545)],
 [12, 825, (140, 180, 220, 35, 80, 50), Team(855120)],
 [13, 805, (190, 170, 190, 20, 60, 45), Team(896972)],
 [14, 795, (110, 200, 100, 65, 35, 65), Team(874317)],
 [15, 790, (60, 150, 90, 25, 110, 70), Team(871787)],
 [16, 785, (90, 150, 120, 60, 20, 100), Team(910344)],
 [17, 780, (185, 180, 190, 95, 45, 25), Team(922608)],
 [18, 760, (140, 110, 220, 45, 50, 40), Team(906259)],
 [19, 730, (210,

In [5]:
gold_partner_teams = max(G.seasons).gold_partner_teams()
silver_partner_teams = max(G.seasons).silver_partner_teams()

In [ ]:
ds = {Te: [nr, prestige] for nr, prestige, ties, Te in standings}

In [15]:
LA = []
for nr, prestige, ties, Te in standings:
    if Te in gold_partner_teams:
        partner = "(G)"
    elif Te in silver_partner_teams:
        partner = "(S)"
    else:
        partner = ""
    LL = [nr, partner, Te, prestige]
    LA.append(LL)

In [16]:
table = texttable.Texttable()
table.set_cols_dtype(["i", "t", "t", "i"])
table.set_cols_align(["r", "c", "l", "r"])
table.set_cols_width([3, 3, 45, 4])
table.set_deco(
    texttable.Texttable.HEADER
    #| texttable.Texttable.VLINES
)
table.add_rows([["Nr", "", "Name", "P"],] + LA)
print(table.draw())

Nr                              Name                         P  
  1   (G)   [867862] Everlasting Vultures of Settra         1095
  2   (S)   [903654] Scions of Neferata                     1020
  3   (G)   [874452] Way of the Dragon                       995
  4   (G)   [876774] Congregation of Holy Touchdown          990
  5   (G)   [886114] Eye and Arrow                           975
  6   (S)   [895929] Fangsnapper's Clawpack                  970
  7   (G)   [874966] Modryn's Midnight Meddlers              935
  8   (S)   [892248] Crimson Fins                            870
  9   (G)   [879131] Sartosa Black Flags                     860
 10   (G)   [878945] Eternal Ecstasy                         855
 11   (G)   [874545] Stone Prince's Men                      830
 12   (G)   [855120] Trantio Dragon Hawks                    825
 13   (S)   [896972] Seekers of Transcendence                805
 14   (G)   [874317] Misty Isle Gators                       795
 15   (S)   [871787] Mali

In [17]:
def tooltips(self):
  if self._tooltips is ...:
    self._tooltips = []
    for sh in self.rostershorthands:
      rosterId = self._rostershorthands[sh]
      roster = sr.roster.Roster(rosterId)
      self._tooltips.append(
          f'[block=tooltip id={sh}]{roster.name}[/block]'
      )
  return self._tooltips

In [18]:
gold_url = "/FUMBBL/Images/gold_small.png"
silver_url = "/FUMBBL/Images/silver_small.png"

In [ ]:
rows = []
tooltip_n = 1
tooltips = []
d_tooltip_n = {}
prev_prestige = None
prev_ties = None
for nr, prestige, ties, Te in standings:
    if ties in d_tooltip_n:
        n = d_tooltip_n[ties]
    else:
        tooltip_n += 1
        n = tooltip_n
        d_tooltip_n[ties] = n
    if (prestige, ties) == (prev_prestige, prev_ties):
        nrstr = ""
    else:
        nrstr = f'{nr}'
    BBprestige = f'[block tooltip={n}]{prestige}[/block]'
    tooltips.append(f'[block=tooltip id={n}]Team prestiges (tiebreaker): {", ".join(str(x) for x in ties)}[/block]\\')
    BBTe = cibblbibbl.bbcode.url(f'/p/team?id={Te.Id}', Te.name)
    if Te in gold_partner_teams:
        BBTe = cibblbibbl.bbcode.img(gold_url) + " " + BBTe
    elif Te in silver_partner_teams:
        BBTe = cibblbibbl.bbcode.img(silver_url) + " " + BBTe
    BBCoach = cibblbibbl.bbcode.url(f'/~{Te.coach_name}', Te.coach_name)
    rows.append([nrstr, BBprestige, BBTe, Te.roster_name, BBCoach])
    prev_prestige = prestige
    prev_ties = ties

In [21]:
align = ["R", "R", "L", "L", "L"]
header = ["#", "Pres.", "Team", "Roster", "Coach"]
width = "600px"

In [22]:
bbcode_table = cibblbibbl.bbcode.table(rows, align=align, header=header, width=width)

In [23]:
s = f'''[block][/block]

[block=automargin width={width}]\\
{bbcode_table}\\
[/block]
{cibblbibbl.bbcode.N.join(tooltips)}

[block][/block]
'''

In [24]:
print(s)

[block][/block]

[block=automargin width=600px]\
[table blackborder border2 width=600px]	[tr bg=black fg=white]		[td][block=center]#[/block][/td]		[td][block=center]Pres.[/block][/td]		[td][block=center]Team[/block][/td]		[td][block=center]Roster[/block][/td]		[td][block=center]Coach[/block][/td]	[/tr]	[tr bg=#e6ddc7]		[td][block=right]1[/block][/td]		[td][block=right][block tooltip=2]1095[/block][/block][/td]		[td][img]/FUMBBL/Images/gold_small.png[/img] [url=/p/team?id=867862]Everlasting Vultures of Settra[/url][/td]		[td]Zandri[/td]		[td][url=/~DatMonsta]DatMonsta[/url][/td]	[/tr]	[tr bg=#d6cdb7]		[td][block=right]2[/block][/td]		[td][block=right][block tooltip=3]1020[/block][/block][/td]		[td][img]/FUMBBL/Images/silver_small.png[/img] [url=/p/team?id=903654]Scions of Neferata[/url][/td]		[td]Lahmian[/td]		[td][url=/~Weresquid]Weresquid[/url][/td]	[/tr]	[tr bg=#e6ddc7]		[td][block=right]3[/block][/td]		[td][block=right][block tooltip=4]995[/block][/block][/td]		[td][img]/FUMBBL/Imag

In [4]:
S

NameError: name 'S' is not defined

In [25]:
Te = Team(897353)

In [26]:
dir(Te)

['Id',
 '_KEY',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__members__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_key',
 '_matchups',
 'achievements',
 'apiget',
 'apimatches',
 'coach_name',
 'config',
 'configfilename',
 'configfilepath',
 'legacyapiget',
 'matches',
 'matchups',
 'name',
 'next_match',
 'next_tournament',
 'players',
 'prev_match',
 'prev_tournament',
 'replays',
 'roster',
 'rosterId',
 'roster_name',
 'search_player',
 'tournaments']

In [29]:
for A in sorted(Te.achievements):
    print(A)

Match Result Prestige([44390] CIBBL - Y3, Summer - Youngblood Scramble, [897353] Court of Shu)
Standings Prestige([44390] CIBBL - Y3, Summer - Youngblood Scramble, [897353] Court of Shu)
Match Result Prestige([44746] CIBBL - Y3, Autumn - Granite Division, [897353] Court of Shu)
Standings Prestige([44746] CIBBL - Y3, Autumn - Granite Division, [897353] Court of Shu)
Match Result Prestige([45219] CIBBL - Y4, Spring - Elemental Planes, [897353] Court of Shu)
Standings Prestige([45219] CIBBL - Y4, Spring - Elemental Planes, [897353] Court of Shu)
Match Result Prestige([45570] CIBBL - Y4, Summer - SCRIBBL Climax Qualifier III, [897353] Court of Shu)
Match Result Prestige([45883] CIBBL - Y4, Autumn - Iron Division, [897353] Court of Shu)
Match Result Prestige([46396] CIBBL - Y5, Spring - Elemental Planes, [897353] Court of Shu)
Standings Prestige([46396] CIBBL - Y5, Spring - Elemental Planes, [897353] Court of Shu)
Match Result Prestige([46707] CIBBL - Y5, Summer - Twin Gates [White], [89735

In [32]:
A.prestige()

140

In [34]:
L = sorted(Te.players)

In [4]:
T = G.tournaments["49185"]

In [5]:
T.standings()

[{'947184': 0}, {'855753': 0}, {'943192': 0}, {'910344': 0}, {'943192': 1, '947184': 0}, {'855753': 0, '910344': 1}, {'855753': 1, '947184': 2}, {'910344': 1, '947184': 2}, {'855753': 2, '943192': 1}, {'910344': 2, '943192': 1}]


[{'cas': 14,
  'casdiff': -1,
  'score': 7,
  'scorediff': 2,
  'td': 7,
  'tddiff': 2,
  'pts': 8,
  'prestige': 80,
  'cto': -1,
  'hth': -1,
  'perf': [('W', 4153050),
   ('D', 4159821),
   ('D', 4172404),
   ('W', 4182625),
   ('L', 4187498)],
  'team': Team(854257),
  'nr': 1},
 {'cas': 14,
  'casdiff': 2,
  'score': 7,
  'scorediff': 0,
  'td': 7,
  'tddiff': 0,
  'pts': 7,
  'prestige': 70,
  'cto': -1,
  'hth': 0,
  'perf': [('W', 4152724),
   ('W', 4166360),
   ('D', 4172404),
   ('L', 4181407),
   ('L', 4189831)],
  'team': Team(943192),
  'nr': 2},
 {'cas': 14,
  'casdiff': 1,
  'score': 5,
  'scorediff': 0,
  'td': 5,
  'tddiff': 0,
  'pts': 7,
  'prestige': 70,
  'cto': -1,
  'hth': 0,
  'perf': [('L', 4152724),
   ('D', 4159821),
   ('W', 4171501),
   ('W', 4180988),
   ('L', 4188150)],
  'team': Team(947184),
  'nr': 3},
 {'cas': 13,
  'casdiff': -1,
  'score': 6,
  'scorediff': 0,
  'td': 6,
  'tddiff': 0,
  'pts': 7,
  'prestige': 70,
  'cto': -112,
  'hth': 0,
  'perf

In [6]:
pts_HTH_results = [
    {'947184': 0}, {'855753': 0}, {'943192': 0}, {'910344': 0},
    {'943192': 1, '947184': 0},
    {'855753': 0, '910344': 1},
    {'855753': 1, '947184': 2},
    {'910344': 1, '947184': 2},
    {'855753': 2, '943192': 1},
    {'910344': 2, '943192': 1}]